In [1]:
#export
"""
This is for quick modifiers, think of them as changing formats
"""
__all__ = ["apply", "applyMp", "applyMpBatched", "applyS",
           "lstrip", "rstrip", "strip",
           "upper", "lower", "replace", "remove", "toFloat", "toInt",
           "sort", "sortF", "consume", "randomize", "stagger"]
from typing import Callable, Iterator, Any, Union, List
from k1lib.cli.init import patchDefaultDelim, BaseCli, settings, T
import k1lib.cli as cli, numpy as np, torch
import multiprocessing as mp; from collections import deque
from functools import partial, update_wrapper
import dill, pickle, k1lib, warnings

In [2]:
#export
def executeFunc(common, line):
    import dill
    f, args, kwargs = dill.loads(common)
    return f(dill.loads(line), *args, **kwargs)
class applyMp(BaseCli):
    _pools = set()
    def __init__(self, f:Callable[[T], T], prefetch:int=None, timeout:float=2, *args, **kwargs):
        """Like :class:`apply`, but execute ``f(row)`` of each row in
multiple processes. Example::

    # returns [3, 2]
    ["abc", "de"] | applyMp(lambda s: len(s)) | deref()
    # returns [5, 6, 9]
    range(3) | applyMp(lambda x, bias: x**2+bias, bias=5) | deref()
    
    # returns [[1, 2, 3], [1, 2, 3]], demonstrating outside vars work
    someList = [1, 2, 3]
    ["abc", "de"] | applyMp(lambda s: someList) | deref()

Internally, this will continuously spawn new jobs up until 80% of all CPU
cores are utilized. On posix systems, the default multiprocessing start method is
``fork()``. This sort of means that all the variables in memory will be copied
over. This might be expensive (might also not, with copy-on-write), so you might
have to think about that. On windows and macos, the default start method is
``spawn``, meaning each child process is a completely new interpreter, so you have
to pass in all required variables and reimport every dependencies. Read more at
https://docs.python.org/3/library/multiprocessing.html#contexts-and-start-methods

If you don't wish to schedule all jobs at once, you can specify a ``prefetch``
amount, and it will only schedule that much jobs ahead of time. Example::

    range(10000) | applyMp(lambda x: x**2)    | head() | deref() # 700ms
    range(10000) | applyMp(lambda x: x**2, 5) | head() | deref() # 300ms

    # demonstrating there're no huge penalties even if we want all results at the same time
    range(10000) | applyMp(lambda x: x**2)    | deref() # 900ms
    range(10000) | applyMp(lambda x: x**2, 5) | deref() # 1000ms

The first line will schedule all jobs at once, and thus will require more RAM and
compute power, even though we discard most of the results anyway (the
:class:`~k1lib.cli.filt.head` cli). The second line only schedules 5 jobs ahead of
time, and thus will be extremely more efficient if you don't need all results right
away.

.. note::

    Remember that every :class:`~k1lib.cli.init.BaseCli` is also a
    function, meaning that you can do stuff like::

        # returns [['ab', 'ac']]
        [["ab", "cd", "ac"]] | applyMp(startswith("a") | deref()) | deref()

    Also remember that the return result of ``f`` should not be a generator.
    That's why in the example above, there's a ``deref()`` inside f.

One last thing. Remember to close all pools (using :meth:`clearPools`) so that
all child processes are terminated, and that resources are freed. Let's say if
you use CUDA tensors, but have not close all pools yet, then it is possible
that CUDA memory is not freed. I learned this the hard way.

:param prefetch: if not specified, schedules all jobs at the same time. If
    specified, schedules jobs so that there'll only be a specified amount of
    jobs, and will only schedule more if results are actually being used.
:param timeout: seconds to wait for job before raising an error
:param args: extra arguments to be passed to the function. ``kwargs`` too"""
        super().__init__(); self.f = f; self.prefetch = prefetch or 1_000_000
        self.timeout = timeout; self.args = args; self.kwargs = kwargs
    def __ror__(self, it:Iterator[T]) -> Iterator[T]:
        super().__ror__(it); it = iter(it) # really make sure it's an iterator, for prefetch
        self.p = p = mp.Pool(mp.cpu_count()*4//5)
        applyMp._pools.add(p); timeout = self.timeout
        common = dill.dumps([self.f, self.args, self.kwargs])
        def gen():
            fs = deque()
            for i, line in zip(range(self.prefetch), it):
                fs.append(p.apply_async(executeFunc, [common, dill.dumps(line)]))
            for line in it:
                yield fs.popleft().get(timeout)
                fs.append(p.apply_async(executeFunc, [common, dill.dumps(line)]))
            for f in fs: yield f.get(timeout)
        return gen()
    @staticmethod
    def clearPools():
        """Terminate all existing pools. Do this before restarting/quitting the
script/notebook to make sure all resources (like GPU) are freed."""
        for p in applyMp._pools:
            try: p.terminate()
            except: pass
        applyMp._pools = set()
    @staticmethod
    def pools():
        """Get set of all pools. Meant for debugging purposes only."""
        return applyMp._pools

In [3]:
import time
assert ["abc", "de"] | applyMp(lambda s: len(s)) | cli.deref() == [3, 2]
assert range(3) | applyMp(lambda x, bias: x**2+bias, bias=5) | cli.deref() == [5, 6, 9]
someList = [1, 2, 3]
assert ["abc", "de"] | applyMp(lambda s: someList) | cli.deref() == [[1, 2, 3], [1, 2, 3]]
assert [["ab", "cd", "ac"]] | applyMp(cli.startswith("a") | cli.deref()) | cli.deref() == [['ab', 'ac']]
beginTime = time.time(); range(10000) | applyMp(lambda x: x**2) | cli.head() | cli.deref(); time1 = time.time() - beginTime
beginTime = time.time(); range(10000) | applyMp(lambda x: x**2, 5) | cli.head() | cli.deref(); time2 = time.time() - beginTime
assert time1 > time2

In [4]:
#export
class applyS(BaseCli):
    def __init__(self, f:Callable[[T], T]):
        """Like :class:`apply`, but much simpler, just operating on the entire input
object, essentially. The "S" stands for "single". Example::

    # returns 5
    3 | applyS(lambda x: x+2)

Like :class:`apply`, you can also use this as a decorator like this::

    @applyS
    def f(x):
        return x+2
    # returns 5
    3 | f

This also decorates the returned object so that it has same qualname, docstring
and whatnot."""
        super().__init__(); self.f = f
        update_wrapper(self, f)
    def __ror__(self, it:T) -> T:
        if settings["useCtx"]: super().__ror__(it)
        return self.f(it)
    def all(self):
        return apply(self.f)

In [5]:
assert 3 | applyS(lambda x: x+2) == 5
@applyS
def f(x): return x+2
assert 3 | f == 5

In [6]:
#export
class apply(BaseCli):
    def __init__(self, f:Callable[[str], str], column:int=None):
        """Applies a function f to every line.
Example::

    # returns [0, 1, 4, 9, 16]
    range(5) | apply(lambda x: x**2) | deref()
    # returns [[3.0, 1.0, 1.0], [3.0, 1.0, 1.0]]
    torch.ones(2, 3) | apply(lambda x: x+2, 0) | deref()

You can also use this as a decorator, like this::

    @apply
    def f(x):
        return x**2
    # returns [0, 1, 4, 9, 16]
    range(5) | f | deref()

:param column: if not None, then applies the function to that column only"""
        super().__init__();
        self.f = f.f if isinstance(f, applyS) else f
        self.column = column
    def __ror__(self, it:Iterator[str]):
        super().__ror__(it); f = self.f; c = self.column
        if c is None: return (f(line) for line in it)
        else: return ([(e if i != c else f(e)) 
                       for i, e in enumerate(row)] for row in it)

In [7]:
assert range(5) | apply(lambda x: x**2) | cli.deref() == [0, 1, 4, 9, 16]
assert torch.ones(2, 3) | apply(lambda x: x+2, 0) | cli.deref() == [[3.0, 1.0, 1.0], [3.0, 1.0, 1.0]]
@apply
def f(x): return x**2
assert range(5) | f | cli.deref() == [0, 1, 4, 9, 16]

In [8]:
#export
def applyMpBatched(f, bs=32, prefetch=2, timeout=5):
    """Pretty much the same as :class:`applyMp` and has the same feel to it
too. Iterator[A] goes in, Iterator[B] goes out, and you specify `f(A) -> B`.
However, this will launch jobs that will execute multiple f(), instead of
1 job per execution. All examples from :class:`applyMp` should work perfectly
here."""
    return cli.batched(bs) | applyMp(apply(f) | cli.deref(True), prefetch, timeout) | cli.joinStreams()

In [9]:
beginTime = time.time(); res1 = range(10000) | applyMp(lambda x: x**2)                     | cli.deref() | cli.head() | cli.deref(); time1 = time.time() - beginTime
beginTime = time.time(); res2 = range(10000) | applyMpBatched(lambda x: x**2, prefetch=10) | cli.deref() | cli.head() | cli.deref(); time2 = time.time() - beginTime
assert time1 > time2; assert res1 == res2

In [10]:
#export
def lstrip(column:int=None, char:str=None):
    """Strips left of every line.
Example::

    # returns ['12 ', '34']
    ["  12 ", " 34"] | lstrip() | deref()"""
    return apply(lambda e: e.lstrip(char), column)
def rstrip(column:int=None, char:str=None):
    """Strips right of every line"""
    return apply(lambda e: e.rstrip(char), column)
def strip(column:int=None, char:str=None):
    """Strips both sides of every line"""
    return apply(lambda e: e.strip(char), column)

In [11]:
assert ["  12 ", " 34"] | lstrip() | cli.deref() == ['12 ', '34']

In [12]:
#export
def upper(column:int=None):
    """Makes all characters uppercase.
Example::

    # returns ['ABCDE', '123R']
    ["abcde", "123r"] | upper() | deref()"""
    return apply(lambda e: e.upper(), column)
def lower(column:int=None):
    """Makes all characters lowercase"""
    return apply(lambda e: e.lower(), column)

In [13]:
assert ["abcde", "123r"] | upper() | cli.deref() == ['ABCDE', '123R']

In [14]:
#export
def replace(s:str, target:str=None, column:int=None):
    """Replaces substring `s` with `target` for each line.
Example::

    # returns ['104', 'ab0c']
    ["1234", "ab23c"] | replace("23", "0") | deref()

:param target: if not specified, then use the default delimiter specified
    in ``cliSettings``"""
    t = patchDefaultDelim(target)
    return apply(lambda e: e.replace(s, t), column)
def remove(s:str, column:int=None):
    """Removes a specific substring in each line."""
    return replace(s, "", column)

In [15]:
assert ["1234", "ab23c"] | replace("23", "0") | cli.deref() == ['104', 'ab0c']

In [16]:
#export
def _op(toOp, c, force, defaultValue):
    return apply(toOp, c) | (apply(lambda x: x or defaultValue, c) if force else (~cli.isValue(None, c)))
def _toFloat(e) -> Union[float, None]:
    try: return float(e)
    except: return None
def toFloat(*columns:List[int], force=False):
    """Converts every row into a float. Example::

    # returns [1, 3, -2.3]
    ["1", "3", "-2.3"] | toFloat() | deref()
    # returns [[1.0, 'a'], [2.3, 'b'], [8.0, 'c']]
    [["1", "a"], ["2.3", "b"], [8, "c"]] | toFloat(0) | deref()

With weird rows::

    # returns [[1.0, 'a'], [8.0, 'c']]
    [["1", "a"], ["c", "b"], [8, "c"]] | toFloat(0) | deref()
    # returns [[1.0, 'a'], [0.0, 'b'], [8.0, 'c']]
    [["1", "a"], ["c", "b"], [8, "c"]] | toFloat(0, force=True) | deref()

:param columns: if nothing, then will convert each row. If available, then
    convert all the specified columns
:param force: if True, forces weird values to 0.0, else filters out all weird rows"""
    if len(columns) > 0:
        return cli.init.serial(*(_op(_toFloat, c, force, 0.0) for c in columns))
    else: return _op(_toFloat, None, force, 0.0)
def _toInt(e) -> Union[int, None]:
    try: return int(float(e))
    except: return None
def toInt(*columns:List[int], force=False):
    """Converts every row into an integer. Example::

    # returns [1, 3, -2]
    ["1", "3", "-2.3"] | toInt() | deref()

:param columns: if nothing, then will convert each row. If available, then
    convert all the specified columns
:param force: if True, forces weird values to 0, else filters out all weird rows

See also: :meth:`toFloat`"""
    if len(columns) > 0:
        return cli.init.serial(*(_op(_toInt, c, force, 0) for c in columns))
    else: return _op(_toInt, None, force, 0)

In [17]:
assert ["1", "3", "-2.3"] | toFloat() | cli.deref() == [1, 3, -2.3]
assert [["1", "a"], ["2.3", "b"], [8, "c"]] | toFloat(0) | cli.deref() == [[1.0, 'a'], [2.3, 'b'], [8.0, 'c']]
assert ["1", "3", "-2.3"] | toInt() | cli.deref() == [1, 3, -2]
assert [["1", "a"], ["c", "b"], [8, "c"]] | toFloat(0, force=True) | cli.deref() == [[1.0, 'a'], [0.0, 'b'], [8.0, 'c']]
assert [["1", "a"], ["c", "b"], [8, "c"]] | toFloat(0) | cli.deref() == [[1.0, 'a'], [8.0, 'c']]

In [18]:
#export
class sort(BaseCli):
    def __init__(self, column:int=0, numeric=True, reverse=False):
        """Sorts all lines based on a specific `column`.
Example::

    # returns [[5, 'a'], [1, 'b']]
    [[1, "b"], [5, "a"]] | ~sort(0) | deref()
    # returns [[2, 3]]
    [[1, "b"], [5, "a"], [2, 3]] | ~sort(1) | deref()
    # errors out, as you can't really compare str with int
    [[1, "b"], [2, 3], [5, "a"]] | sort(1, False) | deref()

:param column: if None, sort rows based on themselves and not an element
:param numeric: whether to convert column to float
:param reverse: False for smaller to bigger, True for bigger to smaller. Use
    :meth:`__invert__` to quickly reverse the order instead of using this param"""
        super().__init__()
        self.column = column; self.reverse = reverse; self.numeric = numeric
        self.filterF = (lambda x: float(x)) if numeric else (lambda x: x)
    def __ror__(self, it:Iterator[str]):
        super().__ror__(it); c = self.column
        if c is None:
            return it | cli.wrapList() | cli.transpose() | sort(0, self.numeric, self.reverse)
        f = self.filterF
        rows = list(it | cli.isNumeric(c) if self.numeric else it)
        def sortF(row):
            if len(row) > c: return f(row[c])
            return float("inf")
        return iter(sorted(rows, key=sortF, reverse=self.reverse))
    def __invert__(self):
        """Creates a clone that has the opposite sort order"""
        return sort(self.column, self.numeric, not self.reverse)

In [19]:
assert [[1, "b"], [5, "a"]] | ~sort(0) | cli.deref() == [[5, 'a'], [1, 'b']]
assert [[1, "b"], [5, "a"], [2, 3]] | ~sort(1) | cli.deref() == [[2, 3]]
try: [[1, "b"], [2, 3], [5, "a"]] | sort(1, False) | cli.deref()
except TypeError: pass

In [20]:
#export
class sortF(BaseCli):
    def __init__(self, f:Callable[[T], float], reverse=False):
        """Sorts rows using a function.
Example::

    # returns ['a', 'aa', 'aaa', 'aaaa', 'aaaaa']
    ["a", "aaa", "aaaaa", "aa", "aaaa"] | sortF(lambda r: len(r)) | deref()
    # returns ['aaaaa', 'aaaa', 'aaa', 'aa', 'a']
    ["a", "aaa", "aaaaa", "aa", "aaaa"] | ~sortF(lambda r: len(r)) | deref()"""
        super().__init__(); self.f = f; self.reverse = reverse
    def __ror__(self, it:Iterator[T]) -> Iterator[T]:
        super().__ror__(it)
        return iter(sorted(list(it), key=self.f, reverse=self.reverse))
    def __invert__(self) -> "sortF":
        return sortF(self.f, not self.reverse)

In [21]:
assert ["a", "aaa", "aaaaa", "aa", "aaaa"] | sortF(lambda r: len(r)) | cli.deref() == ['a', 'aa', 'aaa', 'aaaa', 'aaaaa']
assert ["a", "aaa", "aaaaa", "aa", "aaaa"] | ~sortF(lambda r: len(r)) | cli.deref() == ['aaaaa', 'aaaa', 'aaa', 'aa', 'a']

In [22]:
#export
class consume(BaseCli):
    def __init__(self, f:Union[BaseCli, Callable[[T], None]]):
        r"""Consumes the iterator in a side stream. Returns the iterator.
Kinda like the bash command ``tee``. Example::

    # prints "0\n1\n2" and returns [0, 1, 2]
    range(3) | consume(headOut()) | toList()
    # prints "range(0, 3)" and returns [0, 1, 2]
    range(3) | consume(lambda it: print(it)) | toList()

This is useful whenever you want to mutate something, but don't want to
include the function result into the main stream."""
        super().__init__(); self.f = f
    def __ror__(self, it:T) -> T:
        super().__ror__(it); self.f(it); return it

In [23]:
with k1lib.CaptureStdout() as out:
    assert range(3) | consume(lambda it: print(it)) | cli.toList() == [0, 1, 2]
assert out[0] == "range(0, 3)"
with k1lib.CaptureStdout() as out:
    assert range(3) | consume(cli.headOut()) | cli.toList() == [0, 1, 2]
assert "\n".join(out) == "0\n1\n2"

In [24]:
#export
class randomize(BaseCli):
    def __init__(self, bs=100):
        """Randomize input stream. In order to be efficient, this does not
convert the input iterator to a giant list and yield random values from that.
Instead, this fetches ``bs`` items at a time, randomizes them, returns and
fetch another ``bs`` items. If you want to do the giant list, then just pass
in ``float("inf")``, or ``None``. Example::

    # returns [0, 1, 2, 3, 4], effectively no randomize at all
    range(5) | randomize(1) | deref()
    # returns something like this: [1, 0, 2, 3, 5, 4, 6, 8, 7, 9]. You can clearly see the batches
    range(10) | randomize(3) | deref()
    # returns something like this: [7, 0, 5, 2, 4, 9, 6, 3, 1, 8]
    range(10) | randomize(float("inf")) | deref()
    # same as above
    range(10) | randomize(None) | deref()"""
        super().__init__(); self.bs = bs if bs != None else float("inf")
    def __ror__(self, it:Iterator[T]) -> Iterator[T]:
        super().__ror__(it)
        for batch in it | cli.batched(self.bs, True):
            batch = list(batch); perms = torch.randperm(len(batch))
            for idx in perms: yield batch[idx]

In [25]:
assert range(10) | randomize(1) | cli.deref() == list(range(10))
a = range(10) | randomize(3) | cli.deref()
assert a[-1] == 9; assert len(a) == 10
assert len(range(10) | randomize(float("inf")) | cli.deref()) == 10
assert len(range(10) | randomize(None) | cli.deref()) == 10

In [26]:
#export
class StaggeredStream:
    def __init__(self, stream:Iterator[T], every:int):
        """Not intended to be instantiated by the end user. Use :class:`stagger`
instead."""
        self.stream = stream; self.every = every
    def __iter__(self):
        for i, v in zip(range(self.every), self.stream): yield v
    def __len__(self):
        """Length of window (length of result if you were to deref it)."""
        return self.every
class stagger(BaseCli):
    """Staggers input stream into multiple stream "windows" placed serially. Best
explained with an example::

    o = range(10) | stagger(3)
    o | deref() # returns [0, 1, 2], 1st "window"
    o | deref() # returns [3, 4, 5], 2nd "window"
    o | deref() # returns [6, 7, 8]
    o | deref() # returns [9]
    o | deref() # returns []

This might be useful when you're constructing a data loader::

    dataset = [range(20), range(30, 50)] | transpose()
    dl = dataset | batched(3) | (transpose() | toTensor()).all() | stagger(4)
    for epoch in range(3):
        for xb, yb in dl: # looping over a window
            print(epoch)
            # then something like: model(xb)

The above code will print 6 lines. 4 of them is "0" (because we stagger every 4
batches), and xb's shape' will be (3,) (because we batched every 3 samples).

You should also keep in mind that this doesn't really change the property of the
stream itself. Essentially, treat these pairs of statement as being the same thing::

    o = range(11, 100)
    
    # both returns 11
    o | stagger(20) | item()
    o | item()

    # both returns [11, 12, ..., 20]
    o | head(10) | deref()
    o | stagger(20) | head(10) | deref()

Lastly, multiple iterators might be getting values from the same stream window,
meaning::

    o = range(11, 100) | stagger(10)
    it1 = iter(o); it2 = iter(o)
    next(it1) # returns 11
    next(it2) # returns 12

This may or may not be desirable. Also this should be obvious, but I want to
mention this in case it's not clear to you."""
    def __init__(self, every:int):
        self.every = int(every)
    def __ror__(self, it:Iterator[T]) -> StaggeredStream:
        return StaggeredStream(iter(it), self.every)

In [27]:
o = range(11,100) | stagger(10)
it1 = iter(o); it2 = iter(o)
next(it1), next(it2)

(11, 12)

In [28]:
a = range(10) | stagger(3); assert a | cli.deref() == [0, 1, 2]
assert a | cli.deref() == [3, 4, 5]; assert a | cli.deref() == [6, 7, 8]
assert a | cli.deref() == [9]; assert a | cli.deref() == []
with k1lib.CaptureStdout() as outer:
    dataset = [range(20), range(30, 50)] | cli.transpose()
    dl = dataset | cli.batched(3) | (cli.transpose() | cli.toTensor()).all() | stagger(4)
    for epoch in range(3):
        for xb, yb in dl: print(epoch)
assert outer == ["0"] * 4 + ["1"] * 2
o = range(11, 100)
assert o | stagger(20) | cli.item() == o | cli.item()
assert o | cli.head(10) | cli.deref() == o | stagger(20) | cli.head(10) | cli.deref()

In [30]:
!../../export.py cli/modifier

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.5
Uninstalling k1lib-0.5:
  Successfully uninstalled k1lib-0.5
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/fmt.py -> build/lib/k1lib
copying k1lib/selector.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib